### 手持ち画像ファイルに対する予測を行うノートブック

ライブラリのimport

In [21]:
import os,glob
from PIL import Image
import numpy as np
from keras.utils import np_utils
from keras.models import model_from_json

パラメータ設定（変更したらCellメニュー→Run All Belowを忘れないこと）

In [22]:
w=56
work_dir="data/dogs_cats"
image_dir=work_dir+"/original/unknown"
pred_dir=work_dir+"/predicted"
results_base='dogs_cats-results'
weight_file="dogs_cats-weight-w{0}.hd5".format(w)
model_file="dogs_cats-model-w{0}.json".format(w)
classes=["dogs","cats"]
num_classes=len(classes)

modelファイルの読み込み

In [23]:
json_str=open(model_file,"r").read()
model=model_from_json(json_str)
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
model.load_weights(weight_file)

ファイルひとつずつに対して予測を行う

In [24]:
#結果保存用ファイルを開く
results_file='{0}-w{1}.csv'.format(results_base,w)
res=open(results_file,'w')
res.write("file,{0},{1}\n".format(classes[0],classes[1]))

#出力用ディレクトリを空にする
files=glob.glob(pred_dir+"/*.*")
for f in files:
    os.remove(f)

#ファイルひとつずつに対して予測
files=glob.glob(image_dir+"/*.*")
for f in files:
    #画像データの読み込み
    img=Image.open(f)
    img=img.convert("RGB")
    img=img.resize((w,w))
    #4次元のnumpy配列にreshape
    X=np.array(img).reshape(1,w,w,3).astype("float16")
    X_pred=X/255
    #分類予測
    pred=model.predict(X_pred)
    #画像をpred_dirに出力
    pred_ans=pred.argmax()
    pred_cls=classes[pred_ans]
    tag="pred_as_{0}-w{1}".format(pred_cls,w)
    base=os.path.basename(f)
    img_file="{0}/{1}.{2}.jpg".format(pred_dir,os.path.splitext(base)[0],tag)
    img.save(img_file)
    print(img_file,pred)
    res.write("{0},{1},{2}\n".format(f,pred[0][0],pred[0][1]))

#結果保存用ファイルを閉じる
res.close()

print("OK")

data/dogs_cats/predicted/0.pred_as_dogs-w56.jpg [[0.65410966 0.34589034]]
data/dogs_cats/predicted/1.pred_as_dogs-w56.jpg [[0.62032145 0.3796786 ]]
data/dogs_cats/predicted/2.pred_as_cats-w56.jpg [[0.458485   0.54151493]]
data/dogs_cats/predicted/3.pred_as_dogs-w56.jpg [[0.6427675 0.3572325]]
data/dogs_cats/predicted/4.pred_as_dogs-w56.jpg [[0.99642295 0.00357704]]
data/dogs_cats/predicted/5.pred_as_cats-w56.jpg [[0.16619858 0.8338014 ]]
data/dogs_cats/predicted/6.pred_as_cats-w56.jpg [[0.2529402  0.74705976]]
data/dogs_cats/predicted/7.pred_as_cats-w56.jpg [[0.18634564 0.81365436]]
data/dogs_cats/predicted/8.pred_as_cats-w56.jpg [[0.4100726  0.58992743]]
data/dogs_cats/predicted/9.pred_as_cats-w56.jpg [[0.18018387 0.8198162 ]]
OK
